In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig,
    Trainer
    , TrainingArguments
)
from peft import LoraConfig, PeftModel, LoraConfig, get_peft_model
from trl import SFTTrainer
import os 
from datasets import load_dataset
import pandas as pd

os.environ['HUGGING_FACE_HUB_TOKEN'] = os.getenv("HUGGING_FACE_HUB_TOKEN")

/home/ltnga/ITDSIU21079/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/home/ltnga/ITDSIU21079/src/qwen_v1/checkpoint-100"
model = AutoModelForCausalLM.from_pretrained(model_path)


tokenizer_path = "/home/ltnga/ITDSIU21079/src/qwen_v1/checkpoint-100"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

Loading checkpoint shards: 100%|██████████| 2/2 [00:54<00:00, 27.24s/it]


In [3]:
config = LoraConfig(
    r=32,
    lora_alpha=32,   
    lora_dropout=0.01,
    bias="none",
    use_rslora=True,
    task_type="CAUSAL_LM",
)

In [4]:
model = get_peft_model(model, config)

In [5]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template(examples):
    messages = examples["conversations"]
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

dataset = load_dataset("arcee-ai/EvolKit-20k-vi", split="train")
dataset = dataset.map(apply_template, batched=True)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!


Unsloth: Will map <|im_end|> to EOS = <|endoftext|>.


Model does not have a padding token! Will use pad_token = <|vision_pad|>.


In [6]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2,
        warmup_steps = 4,
        num_train_epochs = 3,
        max_steps = 1000,
        learning_rate = 2e-5,
        fp16 = True,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        save_steps=1000,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "qwen_v2",
        report_to = "none",  
    ))

In [7]:
trainer.train()

Step,Training Loss
1,2.937900
2,2.914400
3,2.984900
4,3.221100
5,2.963000
6,2.738900
7,2.899100
8,3.173700
9,3.078500
10,2.948700


TrainOutput(global_step=1000, training_loss=2.3898793444633486, metrics={'train_runtime': 1759.8756, 'train_samples_per_second': 9.092, 'train_steps_per_second': 0.568, 'total_flos': 2.4554547807657984e+17, 'train_loss': 2.3898793444633486, 'epoch': 1.0395215808632345})